In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def stable_sigmoid1(x):
    # Apply hard thresholds to avoid overflow
    return np.where(x > 20, 1, np.where(x < -20, 0, 1 / (1 + np.exp(-x))))

In [3]:
def stable_sigmoid(x):
    # To prevent overflow, clip x to a safe range
    return np.where(
        x > 20, 1, 
        np.where(x < -20, 0, 1 / (1 + np.exp(-x)))
    )

In [4]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = stable_sigmoid(Z2)
    return Z1, A1, Z2, A2


In [5]:
def backward_propagation(X_batch, y_batch, W1, b1, W2, b2, A1, A2, learning_rate, lambda_reg=0.01):
    m = X_batch.shape[0]
    
    # Compute gradients
    dZ2 = A2 - y_batch.reshape(-1, 1)
    dW2 = np.dot(A1.T, dZ2) / m + lambda_reg * W2
    db2 = np.sum(dZ2) / m
    dZ1 = np.dot(dZ2, W2.T) * A1 * (1 - A1)
    dW1 = np.dot(X_batch.T, dZ1) / m + lambda_reg * W1
    db1 = np.sum(dZ1) / m
    
    # Update weights and biases
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2

In [6]:
def train_nn(X, y, hidden_size, learning_rate, epochs, batch_size=64):
    input_size = X.shape[1]
    output_size = 1
    
    
    W1 = np.random.randn(input_size, hidden_size) * 0.01
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.01
    b2 = np.zeros((1, output_size))
    
    cost_history = []
    accuracy_history = []
    precision_history = []
    recall_history = []
    f1_score_history = []

    for epoch in range(epochs):
        # Forward propagation
        Z1 = np.dot(X, W1) + b1
        A1 = np.tanh(Z1)
        Z2 = np.dot(A1, W2) + b2
        A2 = 1 / (1 + np.exp(-Z2))  # Sigmoid activation

        m = y.shape[0]
        cost = -(1 / m) * np.sum(y * np.log(A2) + (1 - y) * np.log(1 - A2))

        # Backward propagation
        dZ2 = A2 - y.reshape(-1, 1)
        dW2 = (1 / m) * np.dot(A1.T, dZ2)
        db2 = (1 / m) * np.sum(dZ2, axis=0, keepdims=True)
        dA1 = np.dot(dZ2, W2.T)
        dZ1 = dA1 * (1 - np.power(A1, 2))  # Derivative of tanh
        dW1 = (1 / m) * np.dot(X.T, dZ1)
        db1 = (1 / m) * np.sum(dZ1, axis=0, keepdims=True)

        W1 -= learning_rate * dW1
        b1 -= learning_rate * db1
        W2 -= learning_rate * dW2
        b2 -= learning_rate * db2

        y_pred = (A2 > 0.5).astype(int)
        accuracy, precision, recall, f1_score = compute_metrics(y, y_pred)

        cost_history.append(cost)
        accuracy_history.append(accuracy)
        precision_history.append(precision)
        recall_history.append(recall)
        f1_score_history.append(f1_score)

        
        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Cost: {cost:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, F1 Score: {f1_score:.4f}")
    
    return W1, b1, W2, b2, cost_history, accuracy_history, precision_history, recall_history, f1_score_history


In [7]:
def compute_metrics(y_true, y_pred, y_prob):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    # Accuracy
    accuracy = (TP + TN) / (TP + TN + FP + FN)

    # Precision
    precision = TP / (TP + FP) if TP + FP > 0 else 0

    # Recall
    recall = TP / (TP + FN) if TP + FN > 0 else 0

    # F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    # Log Loss
    epsilon = 1e-15
    y_prob = np.clip(y_prob, epsilon, 1 - epsilon)
    log_loss = -np.mean(y_true * np.log(y_prob) + (1 - y_true) * np.log(1 - y_prob))
    return accuracy, precision, recall, f1_score, log_loss

In [8]:
def load_data_from_file(filename):
    # Assuming CSV file format with features and labels in the last column
    data = np.genfromtxt(filename, delimiter=',', skip_header=1)
    X = data[:, :-2]  # All columns except the last two
    y = data[:, -2]   # Binary label is the second-to-last column
    class_labels = data[:, -1]  # Class label is the last column
    return X, y, class_labels

In [9]:
def save_predictions_to_csv(predictions, filename='predictions.csv'):
    # Save the predictions to a CSV file
    np.savetxt(filename, predictions, delimiter=',', header="Predicted Labels", comments='', fmt='%d')
    print(f"Predictions saved to {filename}")

In [10]:
def plot_cost_and_metrics(cost_history, accuracy_history, precision_history, recall_history, f1_score_history):
    epochs = len(cost_history)
    
    plt.figure(figsize=(12, 8))
    
    # Plotting cost
    plt.subplot(2, 2, 1)
    plt.plot(range(epochs), cost_history, label="Cost", color='b')
    plt.xlabel('Epochs')
    plt.ylabel('Cost')
    plt.title('Cost over Epochs')
    plt.legend()
    
    # Plotting accuracy
    plt.subplot(2, 2, 2)
    plt.plot(range(epochs), accuracy_history, label="Accuracy", color='g')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy over Epochs')
    plt.legend()
    
    # Plotting precision
    plt.subplot(2, 2, 3)
    plt.plot(range(epochs), precision_history, label="Precision", color='r')
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.title('Precision over Epochs')
    plt.legend()
    
    # Plotting F1 Score
    plt.subplot(2, 2, 4)
    plt.plot(range(epochs), f1_score_history, label="F1 Score", color='m')
    plt.xlabel('Epochs')
    plt.ylabel('F1 Score')
    plt.title('F1 Score over Epochs')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [11]:
train_file = "nn_train.csv"  # Replace with actual file path
test_file = "nn_test.csv"    # Replace with actual file path

    # Load training and test data
X_train, y_train, train_class_labels = load_data_from_file(train_file)
X_test, y_test, test_class_labels = load_data_from_file(test_file)
print(X_train.shape)

(80000, 1025)


In [12]:
def main():
    # Load train and test data
    train_file = "nn_train.csv"  # Replace with actual file path
    test_file = "nn_test.csv"    # Replace with actual file path

    # Load training and test data
    X_train, y_train, train_class_labels = load_data_from_file(train_file)
    X_test, y_test, test_class_labels = load_data_from_file(test_file)

    # Train the neural network
    hidden_size = 10
    learning_rate = 0.01
    epochs = 1000
    W1, b1, W2, b2, cost_history, accuracy_history, precision_history, recall_history, f1_score_history = train_nn(X_train, y_train, hidden_size, learning_rate, epochs)

    # Predictions on test data
    _, test_predictions = forward_propagation(X_test, W1, b1, W2, b2)
    test_predictions = np.round(test_predictions)
    
    # Compute metrics
    accuracy, precision, recall, f1_score, log_loss = compute_metrics(y_test, test_predictions, test_predictions)

    # Print metrics
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1 Score: {f1_score:.4f}")
    print(f"Test Log Loss: {log_loss:.4f}")
    
    # Save the predicted labels to CSV
    save_predictions_to_csv(test_predictions)

    # Plot cost and metrics
    plot_cost_and_metrics(cost_history, accuracy_history, precision_history, recall_history, f1_score_history)

# Run the main function
if __name__ == "__main__":
    main()

MemoryError: Unable to allocate 47.7 GiB for an array with shape (80000, 80000) and data type float64